<a href="https://colab.research.google.com/github/Benjamon22/Restaurant-Recommendation-Engine/blob/main/Reviews_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_path = '/content/drive/MyDrive/review_no_stopwords.csv'

import pandas as pd
import numpy as np
from scipy.stats import truncnorm
from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
chunks = []
chunksize = 100000

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunks.append(chunk)

# Combine all chunks into one DataFrame
review_df = pd.concat(chunks, ignore_index=True)
review_df.head()

<ipython-input-3-44d0bf205f26>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...


In [4]:
review_df.shape

(4988736, 11)

In [5]:
food_keywords = {"food", "dish", "meal", "taste", "delicious", "menu", "flavor",
                 "cuisine", "plate", "appetizer", "dessert", "drink", "bowl",
                 "burger", "pizza", "soup", "steak", "cheese"
}
service_keywords = {"waiter", "waitress", "staff", "service", "friendly", "rude",
                    "attentive", "host", "manager", "customer", "attitude", "ass", "owner", "option"
}
ambience_keywords = {"ambience", "atmosphere", "environment", "lighting", "music",
                     "decor", "noise", "seating", "vibe", "clean", "interior", "crowded"
}
value_keywords = {
    "value", "price", "worth", "overpriced", "expensive", "cheap", "affordable",
    "cost", "deal", "portion", "serving", "size", "small", "huge", "tiny",
    "filling", "generous", "stingy", "money", "bang", "spent", "enough", "barely"
}

In [6]:
SIA = SentimentIntensityAnalyzer()

In [7]:
tqdm.pandas()
def get_aspect_sentiment(text, keywords):
    if not isinstance(text, str):
        return None
    sentences = text.split('.')
    relevant = [s for s in sentences if any(word in s.lower() for word in keywords)]
    if not relevant:
        return None
    scores = [SIA.polarity_scores(s)['compound'] for s in relevant]
    return sum(scores) / len(scores) if scores else None

review_df['food_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, food_keywords))
review_df['service_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, service_keywords))
review_df['ambience_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, ambience_keywords))
review_df['value_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, value_keywords))
review_df.head()

100%|██████████| 4988736/4988736 [28:03<00:00, 2962.67it/s]


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.6705,0.6705,NaN,0.6705
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.9509,NaN,NaN,NaN
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.9766,0.9766,0.9766,0.9766
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,NaN,0.5859,NaN,NaN
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.9706,NaN,NaN,NaN


In [8]:
review_df['service_sentiment'].min()

-0.9994

In [9]:
review_df['ambience_sentiment'].min()

-0.9993

In [10]:
review_df['food_sentiment'].min()

-0.9993

In [11]:
print(f"Empty values in food_sentiment: {int(review_df['food_sentiment'].isna().sum())}")
print(f"Empty values in service_sentiment: {int(review_df['service_sentiment'].isna().sum())}")
print(f"Empty values in ambience_sentiment: {int(review_df['ambience_sentiment'].isna().sum())}")
print(f"Empty values in value_sentiment: {int(review_df['value_sentiment'].isna().sum())}")

Empty values in food_sentiment: 994939
Empty values in service_sentiment: 2019531
Empty values in ambience_sentiment: 3797032
Empty values in value_sentiment: 3000781


In [12]:
review_original = review_df.copy()
review_original.head(8)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.6705,0.6705,NaN,0.6705
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.9509,NaN,NaN,NaN
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.9766,0.9766,0.9766,0.9766
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,NaN,0.5859,NaN,NaN
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.9706,NaN,NaN,NaN
5,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0.0,0.0,0.0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,0.0,party hibachi waitress brought separate sushi ...,0.9618,0.9618,NaN,0.9618
6,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,4.0,0.0,0.0,0.0,"Locals recommended Milktooth, and it's an amaz...",2015-08-19 14:31:45,0.0,Locals recommended Milktooth amazing jewel Ind...,NaN,NaN,NaN,NaN
7,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0.0,0.0,0.0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,0.0,Love going happy hour dinner Great patio fans ...,0.9382,NaN,NaN,NaN


In [13]:
# Most reviews mentioned about food, but not so much mentioned about ambience or value of food.
# That, however, shows a sign of silence, so we are going to fill the missing sentiments with a normal distribution
def fill_with_truncated_normal(df, column, clip_range=(-0.7, 0.7)):
    valid_values = df[column].dropna()
    mu = valid_values.mean()
    sigma = valid_values.std()
    num_missing = df[column].isna().sum()

    a, b = (clip_range[0] - mu) / sigma, (clip_range[1] - mu) / sigma
    synthetic_values = truncnorm(a, b, loc=mu, scale=sigma).rvs(num_missing)

    df.loc[df[column].isna(), column] = synthetic_values


for col in ['food_sentiment', 'service_sentiment', 'ambience_sentiment', 'value_sentiment']:
    fill_with_truncated_normal(review_df, col)

In [14]:
print(f"Empty values in food_sentiment: {int(review_df['food_sentiment'].isna().sum())}")
print(f"Empty values in service_sentiment: {int(review_df['service_sentiment'].isna().sum())}")
print(f"Empty values in ambience_sentiment: {int(review_df['ambience_sentiment'].isna().sum())}")
print(f"Empty values in value_sentiment: {int(review_df['value_sentiment'].isna().sum())}")

Empty values in food_sentiment: 0
Empty values in service_sentiment: 0
Empty values in ambience_sentiment: 0
Empty values in value_sentiment: 0


In [15]:
review_df.head(8)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.670500,0.670500,0.550811,0.670500
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.950900,0.618136,0.691570,0.414915
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.976600,0.976600,0.976600,0.976600
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,0.566156,0.585900,0.502357,0.600405
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.970600,0.600974,0.644115,0.338325
5,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0.0,0.0,0.0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,0.0,party hibachi waitress brought separate sushi ...,0.961800,0.961800,0.343542,0.961800
6,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,4.0,0.0,0.0,0.0,"Locals recommended Milktooth, and it's an amaz...",2015-08-19 14:31:45,0.0,Locals recommended Milktooth amazing jewel Ind...,0.432505,0.614134,0.679520,0.471717
7,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0.0,0.0,0.0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,0.0,Love going happy hour dinner Great patio fans ...,0.938200,0.553386,0.570254,0.263288


In [16]:
# Some values in the star column has string entries, convert them into floats
review_df['stars'] = pd.to_numeric(review_df['stars'], errors='coerce')
review_df[review_df['stars'].isna()]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
2092131,We were welcomed at the door. The staff here w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.616500,0.263050,0.589678,0.299909
2092132,There were so many great menu choices and a lo...,2016-03-21 19:31:55,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.335203,0.669386,0.640961,0.550507


In [17]:
# Drop the 2 rows with NaN values
review_sorted = review_df.dropna(subset=['stars']).reset_index(drop=True)
review_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988734 entries, 0 to 4988733
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   review_id           object 
 1   user_id             object 
 2   business_id         object 
 3   stars               float64
 4   useful              float64
 5   funny               float64
 6   cool                float64
 7   text                object 
 8   date                object 
 9   review_score        float64
 10  stopwords_removed   object 
 11  food_sentiment      float64
 12  service_sentiment   float64
 13  ambience_sentiment  float64
 14  value_sentiment     float64
dtypes: float64(9), object(6)
memory usage: 570.9+ MB


In [18]:
business_file_path = '/content/drive/MyDrive/business.csv'
business_df = pd.read_csv(business_file_path)
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,hours,open_days,time_span,BusinessAcceptsCreditCards,BusinessParking,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsDelivery,Parking,Parking_Category
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","['Monday', 'Tuesday', 'Wednesday', 'Thursday',...","['7:0-20:0', '7:0-20:0', '7:0-20:0', '7:0-20:0...",False,"{'garage': False, 'street': True, 'validated':...",True,1.0,False,1,Limited Parking
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054.0,40.338183,-75.471659,4.5,13,...,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...","['Wednesday', 'Thursday', 'Friday', 'Saturday'...","['14:0-22:0', '16:0-22:0', '12:0-22:0', '12:0-...",True,"{'garage': None, 'street': None, 'validated': ...",True,NaN,NaN,1,Limited Parking
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015.0,36.269593,-87.058943,2.0,6,...,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","['Monday', 'Tuesday', 'Wednesday', 'Thursday',...","['0:0-0:0', '6:0-22:0', '6:0-22:0', '6:0-22:0'...",True,NaN,True,1.0,True,0,No Parking
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207.0,36.208102,-86.768170,1.5,10,...,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","['Monday', 'Tuesday', 'Wednesday', 'Thursday',...","['0:0-0:0', '6:0-21:0', '6:0-21:0', '6:0-16:0'...",True,"{'garage': False, 'street': False, 'validated'...",True,1.0,True,0,No Parking
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602.0,27.955269,-82.456320,4.0,10,...,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","['Monday', 'Tuesday', 'Wednesday', 'Thursday',...","['11:0-14:0', '11:0-14:0', '11:0-14:0', '11:0-...",NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,0,No Parking


In [19]:
# Find the mean sentiments for each restaurant in the review dataset
avg_sentiments = review_sorted.groupby('business_id')[['food_sentiment', 'service_sentiment', 'ambience_sentiment', 'value_sentiment']].mean().reset_index()
avg_sentiments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56023 entries, 0 to 56022
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_id         56023 non-null  object 
 1   food_sentiment      56023 non-null  float64
 2   service_sentiment   56023 non-null  float64
 3   ambience_sentiment  56023 non-null  float64
 4   value_sentiment     56023 non-null  float64
dtypes: float64(4), object(1)
memory usage: 2.1+ MB


In [20]:
# Merge the mean sentiments to the business dataset
business_original = business_df.copy()
business_df = business_df.merge(avg_sentiments, on='business_id', how='left')
business_df.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,BusinessParking,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsDelivery,Parking,Parking_Category,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,"{'garage': False, 'street': True, 'validated':...",True,1.0,False,1,Limited Parking,0.624360,0.489236,0.532218,0.649538
1,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054.0,40.338183,-75.471659,4.5,13,...,"{'garage': None, 'street': None, 'validated': ...",True,NaN,NaN,1,Limited Parking,0.782169,0.713477,0.672657,0.484677
2,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015.0,36.269593,-87.058943,2.0,6,...,NaN,True,1.0,True,0,No Parking,0.421364,0.183485,0.544797,0.271566
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207.0,36.208102,-86.768170,1.5,10,...,"{'garage': False, 'street': False, 'validated'...",True,1.0,True,0,No Parking,0.307732,0.434920,0.490257,0.334626
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602.0,27.955269,-82.456320,4.0,10,...,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,0,No Parking,0.771255,0.455317,0.451362,0.288777
5,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,"{'garage': None, 'street': False, 'validated':...",True,1.0,True,1,Limited Parking,0.286514,0.222546,0.495166,0.337079
6,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,"{'garage': False, 'street': False, 'validated'...",True,1.0,True,1,Limited Parking,0.688155,0.503949,0.525565,0.647766
7,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,"{u'valet': False, u'garage': None, u'street': ...",True,2.0,True,1,Limited Parking,0.813540,0.647507,0.717619,0.612597
8,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,"{'garage': False, 'street': True, 'validated':...",True,1.0,NaN,1,Limited Parking,0.845271,0.673083,0.563762,0.653600
9,kfNv-JZpuN6TVNSO6hHdkw,Hibachi Express,6625 E 82nd St,Indianapolis,IN,46250.0,39.904320,-86.053080,4.0,20,...,NaN,True,2.0,True,0,No Parking,0.565187,0.516335,0.617454,0.577583


In [21]:
business_df.to_csv('/content/drive/MyDrive/business_with_sentiments.csv', index=False)

In [23]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56023 entries, 0 to 56022
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 56023 non-null  object 
 1   name                        56023 non-null  object 
 2   address                     55629 non-null  object 
 3   city                        56023 non-null  object 
 4   state                       56023 non-null  object 
 5   postal_code                 56014 non-null  float64
 6   latitude                    56023 non-null  float64
 7   longitude                   56023 non-null  float64
 8   stars                       56023 non-null  float64
 9   review_count                56023 non-null  int64  
 10  attributes                  56023 non-null  object 
 11  categories                  56023 non-null  object 
 12  hours                       56023 non-null  object 
 13  open_days                   560

In [ ]:
# We have now completed the sentiment analysis for the review dataset
review_sorted.to_csv('/content/drive/MyDrive/reviews_with_sentiments.csv', index=False)